In [1]:
!pip install html2image

# Start Here

In [57]:
import json
import textwrap
import pytrends
from pytrends.request import TrendReq
import nltk
from nltk.corpus import wordnet
import random
from tqdm import tqdm
from nltk.corpus import wordnet
from nltk.metrics.distance import edit_distance
from nltk.corpus import stopwords
# PlaintextParser
from sumy.parsers.plaintext import PlaintextParser
# LexRankSummarizer
from sumy.summarizers.lex_rank import LexRankSummarizer
# LsaSummarizer
from sumy.summarizers.lsa import LsaSummarizer
# define Tokenizer
from sumy.nlp.tokenizers import Tokenizer
# define Stemmer
from sumy.nlp.stemmers import Stemmer
# define language
from html2image import Html2Image

stopwords = nltk.corpus.stopwords.words('english')


import os
from PIL import Image, ImageDraw, ImageFont
# with open("ppn_deck_cleaned.json", "w") as write_file:
#     json.dump(card_deck, write_file, indent=4)

# read card_deck from ppn_deck.json file
# with open("ppn_deck.json", "r") as read_file:
    # card_deck = json.load(read_file)
with open('ppn_deck.json', 'r') as read_file:
    card_deck = json.load(read_file)

# clear the card_images folder
print("Clearing card_images folder...")
for filename in os.listdir("card_box"):
    os.remove(os.path.join("card_box", filename))



def summarize_text(text, num_sentences):
    """
    Summarize the given text using the LSA or LexRank summarization algorithms and return the summary as a string
    """
    # create a PlaintextParser object to parse the text
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    # choose a summarization algorithm
    # algorithm = LsaSummarizer()
    algorithm = LexRankSummarizer()

    # summarize the text and return the summary as a string
    summary = algorithm(parser.document, num_sentences)
    summary_text = "\n".join([str(sentence) for sentence in summary])

    return summary_text

# def generate_card(title, definition, points, name=None):
#     # if the font is 20 then the max width of the text is 40 characters. Use this to determine how large the title should be.
#     image = Image.new('RGB', (550, 850), (255, 255, 255))
#     draw = ImageDraw.Draw(image)
#     # the title is wrapped to 40 characters, so the height of the rectangle is the number of characters * 20 (the height of the font)
#     title_size = 20
#     font_title = ImageFont.truetype('./fonts/SFNSMono.ttf', title_size)
#     font_description = ImageFont.truetype('./fonts/SFNSMono.ttf', 20)
#     font_points = ImageFont.truetype('./fonts/SFNSMono.ttf', 18)
#     title_wrapped = textwrap.wrap(title, width=40)
#     # draw the title centered horizontally, and 30 pixels from the top. The title is wrapped to 40 characters, so the height of the rectangle is the number of characters * 20 (the height of the font)

#     title_rectangle_height = len(title_wrapped) * 20
#     draw.rectangle([(10, 10), (540, 10 + title_rectangle_height)], fill='lightblue')
#     y_text = 20
#     for line in title_wrapped:
#         draw.text((270, y_text), line, fill=(0, 0, 0), font=font_title, anchor='mm')
#         y_text += 20
#     # draw the definition left justified, and 10 pixels from the the bottom of the title rectangle. The definition is wrapped to 40 characters, so the height of the rectangle is the number of characters / 40 * 20 (the height of the font). 
#     definition_wrapped = textwrap.wrap(definition, width=40)
#     definition_rectangle_height = len(definition_wrapped) * 20
#     draw.rectangle([(10, 10 + title_rectangle_height + 10), (540, 10 + title_rectangle_height + 10 + definition_rectangle_height)], fill='white')
#     y_text = 10 + title_rectangle_height + 20
#     for line in definition_wrapped:
#         draw.text((10, y_text), line, fill=(0, 0, 0), font=font_description, anchor='lm')
#         y_text += 20



#     # draw the points at the bottom of the card centered horizontally, and 20 pixels from the bottom
#     # draw the rectangle around the points, with a light green background, add 10 pixels to the height of the rectangle to make it a little bigger and center the text within the rectangle vertically
#     draw.rectangle([(10, 850 - 30 - 10), (540, 850 - 10)], fill='lightgreen')
#     # draw the text in the rectangle
#     draw.text((270, 850 - 30 - 5), str(points), fill=(0, 0, 0), font=font_points, anchor='mm') # anchor='mm' centers the text horizontally and vertically
#     # save the image with the name of the card if not None
#     if name is not None:
#         image.save('./card_box/{}.png'.format(name)) # save the image with the name of the card, if it's not None
#     else:
#         image.save('./card_box/{}.png'.format(len(os.listdir('./card_box/')))) # save the image with the name of the number of images in the folder

with open('card_html.txt', 'r') as f:
    html_template = f.read()

def generate_card(title: str, description: str, points: str, html_template,  name):
    """
    generate_card generates a card image from the given title, description, and points

    :param title: _description_
    :type title: str
    :param description: _description_
    :type description: str
    :param points: _description_
    :type points: str
    """
    # Create the HTML content for the card
    # The card is 550x850 pixels
    # read the template file from the current directory 'card_html.txt'

    # replace the placeholders with the actual values
    html = html_template
    html = html.replace('Card Title', title)
    html = html.replace('Card Description', description)
    html = html.replace('Point Value', str(points))
    space_to_fill = str(850 - len(description) - 100)
    html = html.replace('MIDDLE_HEIGHT',str(space_to_fill))
    # create blank lines to fill the space
    Middle_Content = ' '*int(space_to_fill)
    html = html.replace('Middle Content',str(Middle_Content))

    # Create an Html2Image object
    hti = Html2Image()

    # Generate a screenshot of the HTML content and save it to a file
    # modified code
    #filename = '{}.png'.format(len(os.listdir('./card_box/')))
    title = title.replace(' ', '_')
    # modified code
    filename = './cards_html/{}.png'.format(title)
    hti.screenshot(html_str=html, save_as=filename)



def generate_physical_cards():
    # Choose a random card to print
    card = random.choice(card_deck)
    print(card)

    # Prepare the summary and points variables
    summary = card['summary']
    if isinstance(summary, list):
        summary = summary[1]
    summary = summarize_text(summary, 2)
    if isinstance(summary, list):
        summary = ' '.join(summary)
    points = len(set(summary.split()))

    # Generate the card
    # With this:
    generate_card(str(card['title']), summary, points=points, html_template=html_template, name=str(card['title']))

    # Iterate through each card and generate a card image for it
    for card in tqdm(card_deck):
        title = card['title']
        summary = card['summary']
        if isinstance(summary, list):
            summary = summary[1]
        summary = summarize_text(summary, 2)
        if isinstance(summary, list):
            summary = ' '.join(summary)
        points = card['point_value']
        name = card['title']
        # Generate the card
        generate_card(str(card['title']), summary, points=points, html_template=html_template, name=name)

print("Initialized process, and ready to generate physical cards...")


Clearing card_images folder...
Initialized process, and ready to generate physical cards...


In [59]:
generate_physical_cards()
print("Done")

{'title': 'Hound Dog (song)', 'summary': 'Hound Dog song, Hound Dog is a twelve-bar blues song written by Jerry Leiber and Mike Stoller. Recorded originally by Big Mama Thornton on August 13, 1952, in Los Angeles and released by Peacock Records in late February 1953, Hound Dog was Thorntons only hit record, selling over 500,000 copies, spending 14 weeks in the R&B charts, including seven weeks at number one. Thorntons recording of Hound Dog is listed as one of the Rock and Roll Hall of Fames 500 Songs That Shaped Rock and Roll, and was inducted into the Grammy Hall of Fame in February 2013.\\nHound Dog has been recorded more than 250 times. The best-known version is the July 1956 recording by Elvis Presley, which ranked number 19 on Rolling Stones list of the 500 Greatest Songs of All Time in 2004, but was excluded from the revised list in 2021; it is also one of the best-selling singles of all time. Presleys version, which sold about 10 million copies globally, was his best-selling so

ValueError: the output_file parameter should be a filename and not a path.
Change the output path by modifying the output_path attribute.